[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1huPsNm9l4OcJvIcu63u0FFWF8X2J7zW3?usp=sharing)

In [10]:
!pip3 install langchain openai langchain_openai langchainhub numexpr

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=28845) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


# Introduction - Fireworks x Langchain

In this notebook, we demonstrate how we can use Fireworks Function Calling model as a router across multiple models with specialized capabilities. Function calling models have seen rapid rise in usage because of their abilities to use external tools easily. One such powerful tool is other LLMs. We have quite a number of specialized OSS LLMs for [Coding](https://www.deepseek.com/), [Chatting in Certain Language](https://github.com/QwenLM/Qwen) or just plain [HF Assistants](https://huggingface.co/chat/assistants).

Function Calling model allows us to
1. Analyze the user query for intent
2. Find the best model to answer the request. And it could be the FC model itself!
3. Construct the right query for the choosen LLM.
4. Profit!

For this notebook we are going to use [LangChain](https://www.langchain.com/) framework to construct an agent chain which is capable of chit chatting & solving math equations using a calculator tool.

This agent chain will take in [custom defined tools](https://python.langchain.com/docs/modules/agents/tools/custom_tools) which are capable of executing a Math Query using a LLM. The main routing LLM is going to be Fireworks function calling model.

## Setup Deps

In order to accomplish the task in this notebook we are goign to import some dependencies from langchain along with LLMMathChain. To read more about what this chain does, check this [documentation](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html).

For solving our math equations, we are going to use the recently released [Mixtral MoE](https://mistral.ai/news/mixtral-of-experts/). For all of our inferencing needs we are going to use the [Fireworks Inference Service](https://fireworks.ai/models).

In order to use the Fireworks AI function calling model, you must first obtain Fireworks API Keys. If you don't already have one, you can one by following the instructions [here](https://readme.fireworks.ai/docs/quickstart). Replace `YOUR_FW_API_KEY` with your obtained key.

**NOTE:** It's important to set temperature to 0.0 for the function calling model because we want reliable behaviour in routing.

In [11]:
import os

from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Optional, Type

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

llm = ChatOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key="YOUR_FW_API_KEY",
    model="accounts/fireworks/models/firefunction-v2",
    temperature=0.0,
    max_tokens=256,
)

math_llm = ChatOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key="fYOUR_FW_API_KEY",
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=0.0,
)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


## Custom Tools

In order to seamlessly use function calling ability of the models, we can use [Custom Tools](https://python.langchain.com/docs/modules/agents/tools/custom_tools) functionality built into LangChain. This allows us to seamlessly define the schema of the functions our model can access along with the execution logic. It simplifies the JSON function specification construction.

For this notebook, we are going to construct a `CustomCalculatorTool` which will use LLM to answer & execute math queries. It takes in a single parameter `query` which has to be valid mathemetical expression.

In [12]:

class CalculatorInput(BaseModel):
    query: str = Field(description="should be a math expression")

class CustomCalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = "Tool to evaluate mathemetical expressions"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(self, query: str) -> str:
        """Use the tool."""
        return LLMMathChain(llm=math_llm, verbose=True).run(query)

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("not support async")

tools = [
  CustomCalculatorTool()
]

agent = create_openai_tools_agent(llm, tools, prompt)

agent = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Test Chit Chat Ability

As we outlined in the beginning, the model should be able to both chit-chat, route queries to external tools when necessary or answer from internal knowledge.

Let's first start with a question that can be answered from internal knowledge.

In [13]:
print(agent.invoke({"input": "What is the capital of USA?"}))



> Entering new AgentExecutor chain...
The capital of the United States of America is Washington D.C.

> Finished chain.
{'input': 'What is the capital of USA?', 'output': 'The capital of the United States of America is Washington D.C.'}


## Test Calculator

Now let's test it's ability to detect a mathematical question & route the query accordingly.

In [14]:
print(agent.invoke({"input": "What is 100 divided by 25?"}))



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'query': '100/25'}`




> Entering new LLMMathChain chain...
100/25

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


```text
100/25
```
...numexpr.evaluate("100/25")...

Answer: 4.0
> Finished chain.
Answer: 4.0The answer is 4.0.

> Finished chain.
{'input': 'What is 100 divided by 25?', 'output': 'The answer is 4.0.'}


# Conclusion

The fireworks function calling model can route request to external tools or internal knowledge appropriately - thus helping developers build co-operative agents.